<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" width="140px" alt="EPFL_logo">

## Image Processing Laboratory Notebooks
---

This Jupyter Notebook is part of a series of computer laboratories designed
to teach image-processing programming; they are running on EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course 
[**MICRO-512 Image Processing II**](https://moodle.epfl.ch/course/view.php?id=522) taught by Prof. M. Unser and Prof. D. Van de Ville.

The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the [Biomedical Imaging Group](http://bigwww.epfl.ch/). 
The distribution or reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2025.

**Authors**: 
    Sébastien Herbreteau, 
    [Daniel Sage](mailto:daniel.sage@epfl.ch), and
    [Sepand Kashani](mailto:sepand.kashani@epfl.ch),
    
---
# Lab 7.2: Introduction to Pyxu
**Released**: Thursday, May 15, 2025

**Submission deadline**: Monday, May 26, 2025, before 23:59 on [Moodle](https://moodle.epfl.ch/course/view.php?id=463)

**Grade weight**: Lab 7 (21 points), 7.5 % of the overall grade

**Related lectures**: Chapter 9 and 10

Double-click on this cell, fill your name and SCIPER number below to verify your identity in Noto and set the seed for random results.
:::{attention} Please write down your name and SCIPER! 
### Student Name: 
### SCIPER:
:::

In [ ]:
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

## Imports
In the next cell, we import Python libraries that  we will use throughout the lab.

In [ ]:
# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import skimage
from interactive_kit import imviewer as viewer

import pyxu
from pyxu.abc import SolverMode
from pyxu.operator import Convolve, DiagonalOp, PositiveL1Norm, SquaredL2Norm, L1Norm, Gradient, PositiveOrthant
from pyxu.opt.solver import PGD
from pyxu.operator.interop import from_source
from pyxu.opt.stop import MaxIter

# Configure plotting as dynamic
%matplotlib widget

# Introduction to Pyxu (7 points)

## 1. Pyxu in a Nutshell (0 points)

[Pyxu](https://pyxu-org.github.io/) is an open source software library to model imaging pipelines and solve inverse problems.
It is based around the core concept of **operators** and **solvers**.

Operators define a transformation from an input space to an output space.
In the context of computational imaging or signal processing, we use the term **forward operator** to describe the chain of transformations that go from the object of interest, i.e. an image-like quantity, to the measured data.
Pyxu comes with a large collection of built-in operators to help build an imaging pipeline.
They can either be used as is, or combined via **operator arithmetic rules** to compose arbitrarily-complex processing pipelines.

```python
>> op1 + op2  # Addition of two operators
>> op1 * op2  # Composition of two operators
>> 4 * op  # Scale operator output
>> op.argscale(alpha)  # Scale operator input
>> op.argshift(shift)  # Shift operator input
>> op.T  # Transpose of (linear) operator
```

#### How to use an operator

Operators are described in Pyxu via instances of ``pyxu.abc.Operator`` or sub-classes thereof.
Their main interface consists of the following information:
- ``dim_shape: tuple[int]``: shape of the input domain.
- ``codim_shape: tuple[int]``: shape of the output domain.
- ``apply(self, arr: ndarray) -> ndarray``: action of the operator.
- ``estimate_lipschitz() -> float``: compute a Lipschitz constant of the operator.

In addition, provided they have the right properties (ex: differentiable, proximable, linear) some of the following methods may also be defined:
- ``grad(self, arr: ndarray) -> ndarray``: gradient of scalar-valued functions.
- ``prox(self, arr: ndarray, tau: float) -> ndarray``: proximity operator of the scaled function $\tau f$.
- ``adjoint(self, arr: ndarray) -> ndarray``: adjoint of the linear operator.
- ``estimate_diff_lipschitz() -> float``: estimate the Lipschitz constant of $\nabla f$.

For example, consider the situation where we have at our disposal a measurement $y$ which is the realization of a forward model $\Phi(s)$, where $s$ is to be determined. If $\Phi$ is a masked convolution, we can express it as the composition of a convolution followed by pixelwise masking:

In [ ]:
kernel = skimage.io.imread('images/psf_movement.tif').astype(np.float64)
# Operator convolution for a 512x512 image
Conv = Convolve(
    dim_shape=(512, 512), 
    kernel=kernel, 
    center=(kernel.shape[0]//2, kernel.shape[1]//2), 
    mode="reflect",
    enable_warnings=False,
)

np.random.seed(1234) # for reproductibility, do not modify
mask = np.random.binomial(n=1, p=0.95, size=(512, 512))
# Operator of partial masking for a 512x512 image
Mask = DiagonalOp(mask)

# Compose operators
Phi = Mask * Conv

The ``Operator.expr()`` method reveals how Pyxu decomposes an operator into a sequence of steps.
This is useful to debug composition problems related to input/output shapes.

In [ ]:
print(Phi.expr())

Thus defined, $\Phi$ is an **operator** which can be applied to any image of size $512 \times 512$, in particular the image from the previous notebook.

In [ ]:
einstein = skimage.io.imread('images/einstein.tif').astype(np.float64)
masked_and_blurred_einstein = Phi(einstein)
np.random.seed(1234) # for reproductibility, do not modify
masked_and_blurred_and_noisy_einstein = masked_and_blurred_einstein + 10 * np.random.randn(*masked_and_blurred_einstein.shape)

plt.close('all')
view = viewer([masked_and_blurred_einstein, masked_and_blurred_and_noisy_einstein, einstein, kernel], widgets=True, hist=False, axis=True, cmap='gray')

#### Loss functions

Inverse problems are specified by loss functions $f: \mathcal{X} \to \mathbb{R}$.
Pyxu defines [common loss functions](https://pyxu-org.github.io/api/operator/func.html) that arise in CI tasks.
Of particular interest for loss functions is the abilitiy to create derived functions using operator arithmetic:

In [ ]:
np.random.seed(1234) # for reproductibility, do not modify
y = masked_and_blurred_and_noisy_einstein
sl2 = SquaredL2Norm(y.shape) # defines the loss \| . \|_2^2
sl2 = sl2.argshift(-y) # defines the loss \| . - y \|_2^2
F = sl2 * Phi # defines the data-fidelity loss \| \Phi(.) - y \|_2^2

print(F.expr())

Pyxu's operator arithmetic also propagates mathematical properties through composition rules.
In the example above, ``F`` is a ``QuadraticFunc`` since it is the composition of a quadratic (``SquaredL2Norm``) and a linear operator (``Phi``).
This ensures that the best computational method is used to perform some steps of operator chains.

#### Regularization terms

To deal with ill-posed problems, it is common practice to add regularization terms to the optimization problem to be solved, which favor certain behaviors of the solution. Once again, regularization terms are also defined by **operators** in Pyxu. For example, if one wants to define a $\ell_1$ norm penalization to promote sparsity with hyperparameter $\lambda$, one would write:

In [ ]:
lam = 1e2
G = lam * L1Norm(y.shape)

print(G.expr())

#### Computing Lipschitz constants

Iterative optimization algorithms often take (scaled) steps $\tau > 0$ in the direction of the function's gradient $\nabla f$.
The step size $\tau$ is often related to the Lipschitz constant of $f$ or $\nabla f$.
Pyxu can compute them for most operator categories, either exactly or provide an upper-bound.
This is achieved via the ``Operator.estimate_lipschitz()`` and ``Operator.estimate_diff_lipschitz()`` functions:

In [ ]:
print("conv Lipschitz (upper bound):", Conv.estimate_lipschitz())  # cheap upper bound
print("conv Lipschitz (exact):", Conv.estimate_lipschitz(method="svd"))  # exact Lipschitz constant

print("F L (upper bound)", F.estimate_diff_lipschitz())
print("F L (exact)", F.estimate_diff_lipschitz(method="svd"))

#### Solvers

[Solver](https://pyxu-org.github.io/api/abc.html#pyxu.abc.Solver) is Pyxu's API to solve inverse problems.

Pyxu offers multiple solvers; an exhaustive list can be found [here](https://pyxu-org.github.io/api/opt.solver.html). Among them is the [**Proximal Gradient Descent (PGD)**](https://pyxu-org.github.io/api/opt.solver.html#pyxu.opt.solver.PGD), which should suit most needs.

**Note:** As mentioned in the [documentation](https://pyxu-org.github.io/api/opt.solver.html#pyxu.opt.solver.PGD), the `PGD` solver expects $f$ to be convex and differentiable with $L$-Lipschitz continuous gradient, and $g$ should be convex, not necessarily differentiable, with a simple proximal operator. This is the case here since the proximal operator of the $\ell_1$ norm can be computed exactly with the soft threshold.

Each solver takes custom inputs in ``__init__()`` and ``fit()``: we refer the reader to the [API reference guide](https://pyxu-org.github.io/api/index.html#pyxu-opt-solver) for details.

Using solvers is a two-stage process:
1. Define the math problem by instantiating a ``Solver``; then
2. Perform the optimization by calling ``Solver.fit()``.

In [ ]:
 # Define the solver
solver = PGD(f=F, g=G, show_progress=False) 

# Solve the optimization problem
dL = F.estimate_diff_lipschitz(method="svd")  # exact Lipschitz constant of \grad F, but time-consuming to obtain
solver.fit(
    x0=np.zeros_like(y),
    tau=1 / dL,
    stop_crit=MaxIter(50),
) # initialization: all-zeros vector; number of iterations fixed to 100

# Get the solution and reshape to the original size
s_opt = solver.solution()

In [ ]:
plt.close('all')
view = viewer([s_opt, y], widgets=True, hist=False, axis=True, cmap='gray')

**Note:** Unsurprisingly, the resulting image is of poor quality as promoting sparsity is not appropriate for this kind of image.

## 2. Hands-on Pyxu with TV regularization (3 points)

$\ell_1$ regularization is not used much with natural images, contrary to TV regularization which favors piecewise-constant solutions. 

For **2 points**, using Pyxu and the **Proximal Gradient Descent** algorithm, compute the solution to the following optimization problem **under positivity constraints**:
$$s_\lambda^\ast = \arg \min_{s \geq \mathbf{0}} \| \Phi(s) - y \|_2^2 + \lambda \| \nabla s \|_2^2$$
where $\lambda=0.05$, $\Phi$ is the function defined in the previous section (convolution with psf of movement followed by partial masking) and $y$ is the blurred, masked and noisy Einstein. Pay attention that, contrary to the previous notebook, the squared $\ell_2$ norm was chosen to ensure differentiability.

**Note:** For reasons of time, stop the algorithm after 50 iterations and take the $y$ vector as initialization (the algorithm should finish in less than 45 seconds).

*Hints:*
    
1. You may want to use the operators [PositiveOrthant](https://pyxu-org.github.io/api/operator/func.html#pyxu.operator.PositiveOrthant) and [Gradient](https://pyxu-org.github.io/api/operator/linop.html#pyxu.operator.Gradient).

2. Be careful with the output dimension of the Gradient operator. 

3. Be careful with the terms dedicated to the `f` argument of the solver `PGD` and those dedicated to the `g` argument.

4. If you think there are convergence issues, consider using the manual iteration mode of the solver, then computing the objective function per iteration.
```python
>>> solver.fit(..., mode=SolverMode.MANUAL)
>>> for data in solver.steps():
>>>     print(solver.objective_func())
```

In [ ]:
lam = 5e-2 # Regularization parameter

# Define the operators
# YOUR CODE HERE

# Define and fit the solver
# YOUR CODE HERE

# Get solution
s_opt = None
# YOUR CODE HERE


# Define and fit the solver
plt.close('all')
view = viewer([s_opt, y], widgets=True, hist=False, axis=True, cmap='gray')

**For 1 point**, answer the following MCQ:
* What is the SNR of the solution found with Pyxu ? (**1 point**)
1. $20.14$ dB
2. $21.14$ dB
3. $22.14$ dB
4. $23.14$ dB
5. $24.14$ dB
6. $25.14$ dB
7. $26.14$ dB 

*Hint:* You may use the next cell to use or redefine the SNR function that you implemented in the first notebook.

In [ ]:
# YOUR CODE HERE

Modify the variable `answer` in the next cell to reflect your choices.

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

## 3. Application: tomographic reconstruction (4 points)

Tomographic reconstruction is a computational process used in imaging sciences to reconstruct 2D or 3D images from a series of projections captured at different angles. 

The fundamental principle behind tomographic reconstruction is the Radon transform, mapping a function in 2D or 3D space to a set of line integrals. In the context of medical imaging, these line integrals represent the attenuation of X-rays as they pass through a human body. By capturing these attenuations from various angles, it is possible to reconstruct a cross-sectional image of the internal structure.

The process typically involves the following steps:

- Data Acquisition: Projection data is gathered from different angles, typically by rotating the X-ray source and detector around the object being imaged. The acquired projection data are then ‘smeared’ back across the imaging field for each angle (back-projection). The data representation that stacks 1D projections of an object at various angles is called a sinogram.

- Reconstruction: Various algorithms, such as Filtered Back-Projection (FBP) or iterative methods, are used to compute the original image from the smeared data. However, due to the non-idealities inherent to the tomographic setup, regularization is often useful to reconstruct the original object.

### Generating ideal sinogram (simulation of real data acquisition)

Given an image (for example `images/MRI.png`), we can simulate the real data acquisition process using the Radon transform (the Pyxu operators `Radon` and its inverse `InvRadon` are given below). In this example, projection data is gathered from 360 angles around the brain. The result is an ideal sinogram.

**Note:** Pyxu has an optimized Radon operator available as an [extension package](https://github.com/pyxu-org/pyxu_xrt)). For simplicity in what follows, we will piggy-back instead on Scikit-Image's [radon()](https://scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.radon) and [iradon()](https://scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.iradon) functions.
Pyxu allows one to easily wrap 3rd-party functions and make them interoperate with its operator machinery.

In [ ]:
# Import brain image
phantom = skimage.io.imread('images/MRI.png').astype(np.float64)[::4, ::4] / 255 # downsampling for speed issues (128x128 image)
nb_angles = 360

In [ ]:
# Radon Operator 
Radon = from_source(cls=pyxu.abc.LinOp,
                    dim_shape=phantom.shape,
                    codim_shape=(phantom.shape[0], nb_angles),
                    apply=lambda _, arr: skimage.transform.radon(image=arr,
                                                                 theta=np.linspace(0, 180, nb_angles),
                                                                 circle=True),
                    adjoint=lambda _, arr: skimage.transform.iradon(radon_image=arr,
                                                                    theta=np.linspace(0, 180, nb_angles),
                                                                    filter_name=None,
                                                                    circle=True),
                    vectorize=["apply", "adjoint"])

# Inverse Radon Operator
InvRadon = from_source(cls=pyxu.abc.LinOp,
                       dim_shape=(phantom.shape[0], nb_angles),
                       codim_shape=phantom.shape,
                       apply=lambda _, arr: skimage.transform.iradon(radon_image=arr,
                                                                     theta=np.linspace(0, 180, nb_angles),
                                                                     filter_name="hamming",
                                                                     circle=True))

In [ ]:
# Compute ideal sinogram
ideal_sino = Radon(phantom)

plt.close('all')
view = viewer([phantom, ideal_sino], widgets=True, hist=False, axis=True, cmap='gray')

### Reconstruction with Filtered Back-Projection (FBP)

In the ideal case, the reconstructed object can be well recovered from its sinogram using the algorithm of Filtered Back-Projection, implemented through the given Pyxu operator `InvRadon`.

In [ ]:
ideal_fbp = InvRadon(ideal_sino)

plt.close('all')
view = viewer([ideal_fbp, phantom, ideal_fbp - phantom], title=["Ideal FBP", "Phantom", "Difference"], widgets=True, hist=False, axis=True, cmap='gray')

### Reconstruction from a non-ideal sinogram

However, in practice, the process of acquisition deviates from the assumptions usually made by traditional Radon transform models. The complexities of the studied setup in this example include:

- Detector Width: Detectors feature non-negligible widths, hence perform tube integrals in place of line integrals assumed by the Radon transform. This results in a blurring effect in the sinogram.

- Probabilistic Element: There’s a chance that detector pixels might malfunction during each scan, introducing missing values in the sinogram.

- X-ray Beam Intensity: The X-ray beam’s intensity tapers towards the edges, causing a similar tapering effect in the sinogram, which complicates the application of the standard Radon transform.

Each of the perturbations are described with Pyxu by the following operators:

In [ ]:
# Detector Width
kernel = np.ones((3, 1)); kernel /= kernel.sum()
Blur1D = Convolve(
    dim_shape=ideal_sino.shape, 
    kernel=kernel, 
    center=(kernel.shape[0]//2, kernel.shape[1]//2), 
    mode="reflect")

# Probabilisic Element
np.random.seed(1234) # for reproductibility, do not modify
mask = np.random.binomial(n=1, p=0.99, size=ideal_sino.shape)
Mask = DiagonalOp(mask)

# X-ray beam intensity
taper = np.hamming(ideal_sino.shape[0])
BeamIntensity = DiagonalOp(dim_shape=ideal_sino.shape, vec=taper.reshape(-1, 1))

# Compose all operators
Phi = BeamIntensity * Mask * Blur1D * Radon

In [ ]:
non_ideal_sino = Phi(phantom)

plt.close('all')
view = viewer([non_ideal_sino, ideal_sino], widgets=True, hist=False, axis=True, cmap='gray')

#### Naive application of Filtered-Back Projection

**For 1 point** answer the following MCQ:
* What is the SNR of naive application of Filtered-Back Projection to the non-ideal sinogram ? (**1 point**)
1. $1.15$ dB
2. $2.15$ dB
3. $3.15$ dB
4. $4.15$ dB
5. $5.15$ dB
6. $6.15$ dB
7. $7.15$ dB

Use the next cell to compute it.

In [ ]:
non_ideal_fbp = None
# YOUR CODE HERE

plt.close('all')
view = viewer([non_ideal_fbp, phantom, non_ideal_fbp - phantom], title=["Non-ideal FBP", "Phantom", "Difference"], widgets=True, hist=False, axis=True, cmap='gray')

Modify the variable `answer` in the next cell to reflect your choices. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

#### Leveraging Pyxu for regularization

For **2 points**, solve the reconstruction problem **under positive constraints**:
$$s^\ast = \arg \min_{s \geq 0} \| \Phi(s) - y \|_2^2 + \lambda  \| s \|_1 + \mu \| \nabla s \|_2^2$$
with $\lambda=1$ and $\mu=10^{-4}$, and where $y$ represents the non-ideal sinogram and $\Phi$ is the composition of the radon transform followed by all the three perturbations defined previously. To do so, use the **Proximal Gradient Descent** solver from Pyxu with gradient step size equal to $10^{-3}$.

**Note:** For reasons of time, stop the algorithm after 200 iterations and take the all-zeros vector as initialization (**takes about 3 minutes**).

*Hints:*
    
1. You may want to use the operator [PositiveL1Norm](https://pyxu-org.github.io/api/operator/func.html#pyxu.operator.PositiveOrthant).

2. The gradient step size is defined with the argument `tau` in the method `fit`.

In [ ]:
y = non_ideal_sino
lam = 1.0 # Regularization parameter
mu = 1e-4 # Regularization parameter

# Define the operators
# YOUR CODE HERE


# Define and fit the solver with all-zeros vector as initialization
# YOUR CODE HERE


# Get solution
s_opt = None
# YOUR CODE HERE

# Display the solution and observe the differences between all the estimators of the phantom image
plt.close('all')
view = viewer([s_opt, non_ideal_fbp, ideal_fbp, phantom], title=["Pyxu solution", "Non-ideal FBP", "Ideal FBP", "Phantom"], widgets=True, hist=False, axis=True, cmap='gray')

For **1 point** answer the following MCQ:
* What is the SNR of the reconstructed image ? (**1 point**)
1. $1.76$ dB
2. $2.76$ dB
3. $3.76$ dB
4. $4.76$ dB
5. $5.76$ dB
6. $6.76$ dB
7. $7.76$ dB

Use the next cell to compute it. 

In [ ]:
# YOUR CODE HERE

Modify the variable `answer` in the next cell to reflect your choices.

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

🎉 Congratulations on finishing the second part of the Inverse Problem lab!! 🎉

Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to Moodle, **in a zip file with the other notebook of this lab.**

* Keep the name of the notebook as: *2_pyxu.ipynb*,
* Name the zip file: *inverse_problem_lab.zip*.